Instalar dependencias (¡SON NECESARIAS!)


In [11]:
!pip install os
!pip install subprocess
!pip install time
!pip install re
!pip install qrcode
!pip install qrcode[pil]
!pip install opencv-python
!pip install pyzbar
!apt install libzbar0


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libzbar0 is already the newest version (0.23.92-4build2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


Código completo con importaciones pertinentes


In [12]:
import os
import cv2
from pyzbar.pyzbar import decode
import qrcode
import re

# Función para generar código QR
def generar_qr(conductor, rut, patente, material, peso, volumen):
    data = f"Conductor: {conductor}\nRUT: {rut}\nPatente: {patente}\nMaterial: {material}\nPeso: {peso}\nVolumen: {volumen}"

    # Encontrar un nombre de archivo único
    numero_qr = 1
    while True:
        nombre_archivo = f"QR{numero_qr}.png"
        if not os.path.exists(nombre_archivo):
            break
        numero_qr += 1

    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(nombre_archivo)

    # Exportar datos a un archivo txt
    with open("datos_camiones.txt", "a") as file:
        file.write(f"Conductor: {conductor}\nRUT: {rut}\nPatente: {patente}\nMaterial: {material}\nPeso: {peso}\nVolumen: {volumen}\n\n")

    return nombre_archivo

# Función para leer código QR
def leer_qr(numero_qr):
    nombre_archivo = f"QR{numero_qr}.png"
    try:
        imagen = cv2.imread(nombre_archivo)
        if imagen is not None:
            d = decode(imagen)
            if d:
                data = d[0].data.decode("utf-8")
                return data
            else:
                print(f"No se pudo leer el código QR en {nombre_archivo}.")
        else:
            print(f"El archivo {nombre_archivo} no existe.")
    except Exception as e:
        print(f"Error al leer el código QR: {e}")
    return None

# Verificación del formato de la patente
def verificar_formato_patente(patente):
    return patente.lower() == 'reemplazo' or (len(patente.split()) == 3 and all(part.isalpha() for part in patente.split()[:-1]) and patente.split()[-1].isdigit())

# Verificación del formato del RUT
def verificar_formato_rut(rut):
    rut_pattern = re.compile(r'^\d{1,3}(\.\d{3}){0,2}-[\dkK]$')
    return bool(rut_pattern.match(rut))

# Ingreso manual de datos del camión
while True:
    try:
        conductor = input("Ingrese el nombre del conductor: ")
        rut = input("Ingrese el RUT del conductor (Formato: 123.456.789-K): ")

        while not verificar_formato_rut(rut):
            print("Formato de RUT incorrecto. Por favor, vuelva a ingresar el RUT.")
            rut = input("Ingrese el RUT del conductor (Formato: 123.456.789-K): ")

        break

    except ValueError as ve:
        print(f"Ingrese un valor válido. {ve}")

while True:
    try:
        patente = input("Ingrese la patente del camión (Ejemplo formato: DR KL 25) en caso de ser reemplazo indique 'Reemplazo': ")
        if verificar_formato_patente(patente):
            break
        else:
            raise ValueError("Formato de patente incorrecto. Por favor, vuelva a ingresar la patente.")
    except ValueError as ve:
        print(f"Ingrese un valor válido. {ve}")

material = input("Ingrese el material transportado: ")
peso = input("Ingrese el peso del camión: ")

# En caso de ser reemplazo, el volumen debe quedar vacío
if patente.lower() == 'reemplazo':
    volumen = ''
else:
    while True:
        try:
            volumen = input("Ingrese el volumen en m3: ")
            if not volumen.isdigit():
                raise ValueError("Ingrese un valor numérico para el volumen.")
            break
        except ValueError as ve:
            print(f"Ingrese un valor válido. {ve}")

# Generar código QR único para el camión
try:
    nombre_qr = generar_qr(conductor, rut, patente, material, peso, volumen)
    print(f"\nCódigo QR generado para el camión {patente}.")
    print(f"Nombre del archivo QR: {nombre_qr}")
except Exception as e:
    print(f"Error al generar el código QR: {e}")
    exit(1)

# Segunda etapa - Leer código QR y recuperar datos del camión
numero_qr_escaneado = input("Ingrese el número del archivo QR a escanear (por ejemplo, 1): ")
datos_camion = leer_qr(int(numero_qr_escaneado))

# Imprimir los datos recuperados del camión
if datos_camion:
    print("\nDatos del camión recuperados del código QR:")
    for linea in datos_camion.split('\n'):
        print(linea)

# Mostrar datos en consola desde el archivo txt


print(f"\nArchivos QR y txt creados correctamente.")


Ingrese el nombre del conductor: Cristobal
Ingrese el RUT del conductor (Formato: 123.456.789-K): 21.725.621-2
Ingrese la patente del camión (Ejemplo formato: DR KL 25) en caso de ser reemplazo indique 'Reemplazo': DR KL 25
Ingrese el material transportado: Acero
Ingrese el peso del camión: 23
Ingrese el volumen en m3: 12

Código QR generado para el camión DR KL 25.
Nombre del archivo QR: QR1.png
Ingrese el número del archivo QR a escanear (por ejemplo, 1): 1

Datos del camión recuperados del código QR:
Conductor: Cristobal
RUT: 21.725.621-2
Patente: DR KL 25
Material: Acero
Peso: 23
Volumen: 12

Archivos QR y txt creados correctamente.
